In [1]:
!wget https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_collection_data.pkl
!wget https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_query_tweets_dev.tsv

--2025-04-19 10:19:09--  https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_collection_data.pkl
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16262100 (16M) [application/octet-stream]
Saving to: ‘subtask4b_collection_data.pkl’

subtask4b_collectio 100%[===================>]  15.51M  19.6MB/s    in 0.8s    

2025-04-19 10:19:10 (19.6 MB/s) - ‘subtask4b_collection_data.pkl’ saved [16262100/16262100]

--2025-04-19 10:19:11--  https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_query_tweets_dev.tsv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298209 (291K) [text/plain]
Saving t

In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Load the document base
with open('subtask4b_collection_data.pkl', 'rb') as f:
    collection_data = pickle.load(f)

# Load the query set
query_data = pd.read_csv('subtask4b_query_tweets_dev.tsv', sep='\t')

<ipython-input-3-eceb919f7c0e>:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  collection_data = pickle.load(f)


In [4]:
query_data.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [5]:
# Extract document texts from the collection data
doc_texts = [data['title'] + ' ' + data['abstract'] for _, data in collection_data.iterrows()]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer to the document texts
doc_matrix = vectorizer.fit_transform(doc_texts)

In [7]:
results = []

for index, row in query_data.iterrows():
    tweet_id = row['post_id']
    tweet_text = row['tweet_text']

    # Transform the query tweet into a vector
    query_vec = vectorizer.transform([tweet_text])

    # Calculate cosine similarity between the query and all documents
    similarities = cosine_similarity(query_vec, doc_matrix).flatten()

    # Get the indices of the top 5 most similar documents
    top_indices = np.argsort(similarities)[::-1][:5]

    # Get the document IDs corresponding to the top indices
    # Fix: Use iloc to access elements by position
    top_doc_ids = [collection_data.iloc[i]['cord_uid'] for i in top_indices]

    # Directly assign the predicted doc_ids to the DataFrame
    # query_data.at[index, 'predicted_docs'] = top_doc_ids
    results.append({
        "id": tweet_id,
        "doc_ids": top_doc_ids
    })

In [9]:
df = pd.DataFrame(results)

In [15]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [16]:
get_performance_mrr(pd.merge(query_data, df, left_on='post_id', right_on='id', how='left'), 'cord_uid', 'doc_ids')

{1: np.float64(0.4635714285714286),
 5: np.float64(0.52925),
 10: np.float64(0.52925)}

In [12]:
pd.merge(query_data, df, left_on='post_id', right_on='id', how='left')

,post_id,tweet_text,cord_uid,predicted_docs,id,doc_ids
0,16,covid recovery: this study from the usa reveal...,3qvh482o,nan,16,"[3qvh482o, 59up4v56, wfa5fb3r, 66g5lpm6, dcfqv..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,nan,69,"[r58aohnu, yrowv62k, rpwmcqbg, p0kg6dyz, iu1d9..."
2,73,I recall early on reading that researchers who...,sts48u9i,nan,73,"[h6gdck2u, acfa95vw, whh34bjb, gruir7aw, gjtef..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,nan,93,"[3sr2exq9, z795y51f, sv48gjkk, fx2ikn2t, k0f4c..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,nan,96,"[ybwwmyqy, 3l6ipiwk, vabb2f26, qh6rif48, wdw0a..."
...,...,...,...,...,...,...
1395,14193,Residents at high risk of covid-19: effectiven...,0gn3b98n,nan,14193,"[0e97tdez, vjvyjblu, xs7vkm19, wotf0lzx, 56rn6..."
1396,14196,"61% of teenagers hospitalized for covid were ""...",25bdifv6,nan,14196,"[iz6nrfpc, yhmcx7ae, zjbyz1vy, bbs7zocu, 1tqwm..."
1397,14203,"""fresh evidence backing melatonin against covi...",qn6wawxk,nan,14203,"[qn6wawxk, 059oj76m, uy1xibll, t8jlihgm, 7x1aj..."
1398,14233,"the vaccine doesn't halt the spread, it is pro...",3u3i5myh,nan,14233,"[zrnxq5uq, wt6azxc1, ky5env7t, e63j8gci, wzqdo..."


In [14]:
# Print the 'results' DataFrame to a CSV file
df.to_csv('results.tsv', index=None, sep='\t', columns=['id', 'doc_ids'])

In [ ]:
import json

with open("submission.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ TF-IDF submission saved as submission.json")

✅ TF-IDF submission saved as submission.json
